In [24]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

import rtbm.layers as layers
import rtbm.model as mdl

import warnings
warnings.filterwarnings('ignore')

from rtbm.costfunctions import mse, crossentropy
from rtbm.activations import tanh, sigmoid
from rtbm.initializers import uniform, normal
from rtbm.gradientschemes import adam, RMSprop, adagrad, adadelta
from rtbm import minimizer

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelBinarizer

from scipy import signal

from sklearn import datasets
from sklearn.model_selection import train_test_split

# Example 3: IRIS (via CMA)

In [25]:
# Load MNIST dataset
iris = datasets.load_iris()


X_train, X_test, Y_train, Y_test = train_test_split(iris.data, iris.target, test_size=0.4, random_state=0)


enc = LabelBinarizer()
enc.fit(np.diag([1,1,1]))
enc.classes_ = [0, 1, 2]

T=enc.transform(Y_train).T

In [191]:
M = mdl.Model()
M.add(layers.DiagExpectationUnitLayer(4,3, param_bound=20))
M.add(layers.NonLinear(3,3))

In [31]:
minim = minimizer.CMA(False)

sol=minim.train(mse(), M, np.transpose(X_train), T, maxiter=500)

CMA on 1 cpu(s) enabled
(7_w,14)-aCMA-ES (mu_w=4.3,w_1=36%) in dimension 30 (seed=464686, Mon Dec  4 10:56:24 2017)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     14 1.516037997454532e+00 1.0e+00 1.89e+00  2e+00  2e+00 0:00.0
    2     28 1.641732212617864e+00 1.1e+00 1.85e+00  2e+00  2e+00 0:00.1
    3     42 1.693121385357492e+00 1.1e+00 1.86e+00  2e+00  2e+00 0:00.2
   42    588 1.519697530742598e+00 1.7e+00 1.18e+00  1e+00  1e+00 0:03.2
   95   1330 1.011556835155225e+00 2.4e+00 5.80e-01  5e-01  7e-01 0:07.2
  100   1400 1.177030084924074e+00 2.4e+00 5.33e-01  4e-01  6e-01 0:07.6
  184   2576 3.398811107501550e-01 3.6e+00 2.90e-01  2e-01  3e-01 0:13.7
  200   2800 3.361501182564704e-01 3.9e+00 2.79e-01  2e-01  3e-01 0:14.8
  300   4200 9.167088142514501e-02 6.3e+00 1.02e-01  6e-02  1e-01 0:22.0
  400   5600 6.014728151637073e-02 9.8e+00 7.99e-02  4e-02  1e-01 0:29.2
  500   7000 4.615312613812557e-02 2.5e+01 6.03e-02  3e-02  1e-01 0:36.4
CMAEvolut

In [32]:
P=np.argmax(M.predict(np.transpose(X_train)),axis=0)

print(classification_report(Y_train,P.T))
print(confusion_matrix(Y_train, P.T))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        34
          1       1.00      0.96      0.98        27
          2       0.97      1.00      0.98        29

avg / total       0.99      0.99      0.99        90

[[34  0  0]
 [ 0 26  1]
 [ 0  0 29]]


In [33]:
P=np.argmax(M.predict(np.transpose(X_test)),axis=0)


print(classification_report(Y_test,P.T))
print(confusion_matrix(Y_test, P.T))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        16
          1       0.92      0.96      0.94        23
          2       0.95      0.90      0.93        21

avg / total       0.95      0.95      0.95        60

[[16  0  0]
 [ 0 22  1]
 [ 0  2 19]]


# Example 4: IRIS (via SGD)

In [207]:
M = mdl.Model()
#M.add(layers.NonLinear(4,3,tanh))
M.add(layers.DiagExpectationUnitLayer(4,3, param_bound=20))
#M.add(layers.NonLinear(2,3))

In [ ]:
minim = minimizer.SGD()

sol =minim.train(mse, M, np.transpose(X_train), T,scheme=adam(), lr=0.001, maxiter=10000)

In [ ]:
P=np.argmax(M.predict(np.transpose(X_train)),axis=0)

print(classification_report(Y_train,P.T))
print(confusion_matrix(Y_train, P.T))

In [ ]:
P=np.argmax(M.predict(np.transpose(X_test)),axis=0)


print(classification_report(Y_test,P.T))
print(confusion_matrix(Y_test, P.T))

In [ ]:
M.get_layer(1).show_activation(1,1)
M.get_layer(1).show_activation(2,1)
M.get_layer(1).show_activation(3,1)
